In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import warnings
from itertools import product
from datetime import datetime
warnings.filterwarnings('ignore')
plt.style.use('seaborn-poster')

In [ ]:
df = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv')
df.head(10)

In [ ]:
print(df.Timestamp.head())
df.Timestamp = pd.to_datetime(df.Timestamp, unit='s')
print(df.Timestamp.head())

In [ ]:
# Resampling to daily frequency
df.index = df.Timestamp
df = df.resample('D').mean()
df.head()

In [ ]:
# Resampling to monthly frequency
df_month = df.resample('M').mean()

# Resampling to annual frequency
df_year = df.resample('A-DEC').mean()

# Resampling to quarterly frequency
df_Q = df.resample('Q-DEC').mean()

In [ ]:
# PLOTS
fig = plt.figure(figsize=[15, 7])
plt.suptitle('Bitcoin exchanges, mean USD', fontsize=22)

plt.subplot(221)
plt.plot(df.Weighted_Price, '-', label='By Days')
plt.legend()

plt.subplot(222)
plt.plot(df_month.Weighted_Price, '-', label='By Months')
plt.legend()

plt.subplot(223)
plt.plot(df_Q.Weighted_Price, '-', label='By Quarters')
plt.legend()

plt.subplot(224)
plt.plot(df_year.Weighted_Price, '-', label='By Years')
plt.legend()

# plt.tight_layout()
plt.show()

In [ ]:
# df_month=df_month.drop('month',1)

In [ ]:
# df_month.index
# df['month']=pd.DataFrame(np.arange(len(df_month.index)))
# df['month']

df_month['indep']=np.arange(len(df_month.index)).tolist()
df_month.head(10)

In [ ]:
#Linear Regression

X=df_month['indep']
Y=df_month['Weighted_Price']
# X = sm.add_constant(X)
model = sm.OLS(Y,sm.add_constant(X))
results = model.fit()
plt.scatter(X,Y,alpha=0.3)
y_predict = results.params[0] + results.params[1]*df_month['indep']
plt.plot(X,y_predict, linewidth=3)
# plt.xlim(240,350)
# plt.ylim(100,350)
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('OLS Regression')
print(results.summary())
# results.params


In [ ]:

X

In [ ]:
# POlynomial Regression
X=df_month['indep']
Y=df_month['Weighted_Price']
mymodel = np.poly1d(np.polyfit(X, Y, 3))

myline = np.linspace(1, 106, 106)

plt.scatter(X, Y)
plt.plot(myline, mymodel(myline))
plt.show()

In [ ]:
# Normality Test
X=df_month['indep']
Y=df_month['Weighted_Price']
# X = sm.add_constant(X)
model = sm.OLS(Y,sm.add_constant(X))
results = model.fit()
plt.scatter(X,Y,alpha=0.3)
y_predict = results.params[0] + results.params[1]*df_month['indep']
residual=df_month['Weighted_Price']-y_predict

print('p value of Jarque-Bera test is: ', stats.jarque_bera(residual)[1])
print('p value of Shapiro-Wilk test is: ', stats.shapiro(residual)[1])
print('p value of Kolmogorov-Smirnov test is: ', stats.kstest(residual, 'norm')[1])
# plt.plot(X,y_predict, linewidth=3)
# plt.xlim(240,350)
# plt.ylim(100,350)
# plt.xlabel('Time')
# plt.ylabel('Price')
# plt.title('OLS Regression')
# print(results.summary())
# results.params

In [ ]:
# Homogeneity Test
print('p value of Goldfeld–Quandt test is: ', sms.het_goldfeldquandt(results.resid, results.model.exog)[1])
print('p value of Breusch–Pagan test is: ', sms.het_breuschpagan(results.resid, results.model.exog)[1])
print('p value of White test is: ', sms.het_white(results.resid, results.model.exog)[1])

In [ ]:
# Stationarity Test
print('The Durbin-Watson statistic is: ', sms.durbin_watson(results.resid))    #Positive Serial Correlation because its close to 0
print('p value of Breusch-Godfrey test is: ', sms.acorr_breusch_godfrey(results,nlags=1)[3])